In [ ]:
# Install dependencies
!pip install -q transformers>=4.30.0 mediapipe

In [ ]:
import os
import sys
from pathlib import Path

# Set working directory to /kaggle/working
os.chdir('/kaggle/working')

# Configuration
CONFIG = {
    'data_dir': '/kaggle/input/wlalsl-with-landmarks/data/wlasl',  # Changed
    'landmarks_dir': '/kaggle/input/wlalsl-with-landmarks/data/wlasl/landmarks',  # Changed
    'checkpoint_dir': '/kaggle/working/checkpoints',
    'subset': 'nslt_300.json',
    'batch_size': 4,
    'num_epochs': 50,
    'num_frames': 16,
    'hidden_dim': 256,
    'learning_rate': 1e-4,
    'fusion_type': 'concat',
    'device': 'cuda',
}
print("Configuration:")
for k, v in CONFIG.items():
    print(f"  {k}: {v}")

In [ ]:
# Download MediaPipe models
import urllib.request

def download_mediapipe_models():
    """Download MediaPipe task models to working directory."""
    models = {
        'hand_landmarker.task': 'https://storage.googleapis.com/mediapipe-models/hand_landmarker/hand_landmarker/float16/1/hand_landmarker.task',
        'pose_landmarker.task': 'https://storage.googleapis.com/mediapipe-models/pose_landmarker/pose_landmarker_heavy/float16/1/pose_landmarker_heavy.task'
    }
    
    for filename, url in models.items():
        if not Path(filename).exists():
            print(f"Downloading {filename}...")
            urllib.request.urlretrieve(url, filename)
            print(f"✓ Downloaded {filename}")
        else:
            print(f"✓ {filename} already exists")

download_mediapipe_models()

In [ ]:
# Copy source files from input dataset if using custom dataset structure
# If your dataset includes the python files, copy them:
source_files = ['hybrid_asl_model.py', 'train_hybrid_asl.py']
for f in source_files:
    src = Path(f'/kaggle/input/wlasl/{f}')
    dst = Path(f'/kaggle/working/{f}')
    if src.exists() and not dst.exists():
        import shutil
        shutil.copy(src, dst)
        print(f"Copied {f}")

In [ ]:
# Import libraries
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from transformers import VideoMAEModel, VideoMAEImageProcessor
import numpy as np
import json
import cv2

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# MODEL DEFINITIONS (inline for Kaggle notebook portability)
# ═══════════════════════════════════════════════════════════════════════════════

class LandmarkEncoder(nn.Module):
    """Transformer encoder for landmark sequences."""
    
    def __init__(self, input_dim=162, hidden_dim=256, num_heads=4, num_layers=2, dropout=0.3):
        super().__init__()
        
        self.input_projection = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        self.pos_encoding = nn.Parameter(torch.randn(1, 64, hidden_dim) * 0.02)
        
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim * 4,
            dropout=dropout,
            batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        
        self.output_projection = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU()
        )
        
        self.hidden_dim = hidden_dim
        
    def forward(self, x):
        batch_size, seq_len, _ = x.shape
        x = self.input_projection(x)
        x = x + self.pos_encoding[:, :seq_len, :]
        x = self.transformer(x)
        x = x.mean(dim=1)
        x = self.output_projection(x)
        return x


class VideoMAEEncoder(nn.Module):
    """VideoMAE encoder for visual features."""
    
    def __init__(self, model_name='MCG-NJU/videomae-base', hidden_dim=256, freeze_backbone=False, dropout=0.3):
        super().__init__()
        
        self.videomae = VideoMAEModel.from_pretrained(model_name)
        
        if freeze_backbone:
            for param in self.videomae.parameters():
                param.requires_grad = False
        
        videomae_hidden = self.videomae.config.hidden_size
        
        self.projection = nn.Sequential(
            nn.Linear(videomae_hidden, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout)
        )
        
        self.hidden_dim = hidden_dim
        
    def forward(self, pixel_values):
        outputs = self.videomae(pixel_values=pixel_values)
        hidden_states = outputs.last_hidden_state
        pooled = hidden_states.mean(dim=1)
        features = self.projection(pooled)
        return features


class HybridASLModel(nn.Module):
    """Hybrid model combining VideoMAE and MediaPipe landmarks."""
    
    def __init__(self, num_classes, videomae_model='MCG-NJU/videomae-base', hidden_dim=256,
                 freeze_videomae=False, dropout=0.3, fusion_type='concat'):
        super().__init__()
        
        self.fusion_type = fusion_type
        self.hidden_dim = hidden_dim
        
        self.visual_encoder = VideoMAEEncoder(
            model_name=videomae_model,
            hidden_dim=hidden_dim,
            freeze_backbone=freeze_videomae,
            dropout=dropout
        )
        
        self.landmark_encoder = LandmarkEncoder(
            input_dim=162,
            hidden_dim=hidden_dim,
            num_heads=4,
            num_layers=2,
            dropout=dropout
        )
        
        if fusion_type == 'concat':
            fusion_input_dim = hidden_dim * 2
            self.fusion = nn.Sequential(
                nn.Linear(fusion_input_dim, hidden_dim),
                nn.LayerNorm(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            )
        elif fusion_type == 'attention':
            self.fusion_attention = nn.MultiheadAttention(
                embed_dim=hidden_dim,
                num_heads=4,
                dropout=dropout,
                batch_first=True
            )
            self.fusion = nn.Sequential(
                nn.Linear(hidden_dim * 2, hidden_dim),
                nn.LayerNorm(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            )
        elif fusion_type == 'gated':
            self.gate = nn.Sequential(
                nn.Linear(hidden_dim * 2, hidden_dim),
                nn.Sigmoid()
            )
            self.fusion = nn.Sequential(
                nn.Linear(hidden_dim, hidden_dim),
                nn.LayerNorm(hidden_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            )
        
        self.classifier = nn.Sequential(
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_classes)
        )
        
        self.num_classes = num_classes
        
    def forward(self, pixel_values, landmarks):
        visual_features = self.visual_encoder(pixel_values)
        landmark_features = self.landmark_encoder(landmarks)
        
        if self.fusion_type == 'concat':
            fused = torch.cat([visual_features, landmark_features], dim=-1)
            fused = self.fusion(fused)
        elif self.fusion_type == 'attention':
            v_expanded = visual_features.unsqueeze(1)
            l_expanded = landmark_features.unsqueeze(1)
            attended_v, _ = self.fusion_attention(v_expanded, l_expanded, l_expanded)
            attended_l, _ = self.fusion_attention(l_expanded, v_expanded, v_expanded)
            fused = torch.cat([attended_v.squeeze(1), attended_l.squeeze(1)], dim=-1)
            fused = self.fusion(fused)
        elif self.fusion_type == 'gated':
            combined = torch.cat([visual_features, landmark_features], dim=-1)
            gate = self.gate(combined)
            fused = gate * visual_features + (1 - gate) * landmark_features
            fused = self.fusion(fused)
        
        logits = self.classifier(fused)
        return logits
    
    def predict(self, pixel_values, landmarks):
        logits = self.forward(pixel_values, landmarks)
        probs = F.softmax(logits, dim=-1)
        confidence, predictions = probs.max(dim=-1)
        return predictions, confidence, probs

print("✓ Model classes defined")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# DATASET CLASS WITH LANDMARK CACHING AND FRAME TRIMMING
# ═══════════════════════════════════════════════════════════════════════════════

class HybridASLDataset(torch.utils.data.Dataset):
    """Dataset with pre-extracted landmark support and frame trimming."""
    
    def __init__(self, video_paths, labels, videomae_processor, num_frames=16, 
                 image_size=224, landmarks_dir=None, frame_info=None):
        self.video_paths = video_paths
        self.labels = labels
        self.videomae_processor = videomae_processor
        self.num_frames = num_frames
        self.image_size = image_size
        self.landmarks_dir = Path(landmarks_dir) if landmarks_dir else None
        self.frame_info = frame_info  # (start_frame, end_frame) for each video
        
        # Build landmarks cache lookup
        self.cached_landmarks = {}
        if self.landmarks_dir:
            for vp in video_paths:
                video_id = Path(vp).stem
                landmark_path = self.landmarks_dir / f"{video_id}_landmarks.npy"
                if landmark_path.exists():
                    self.cached_landmarks[video_id] = str(landmark_path)
            print(f"  Found {len(self.cached_landmarks)}/{len(video_paths)} pre-extracted landmarks")
        
    def __len__(self):
        return len(self.video_paths)
    
    def load_video_frames(self, video_path, start_frame=None, end_frame=None):
        """Load video frames with optional frame trimming."""
        cap = cv2.VideoCapture(str(video_path))
        frames = []
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        # Apply frame trimming if specified (convert 1-indexed to 0-indexed)
        if start_frame is not None and end_frame is not None:
            frame_start = max(0, start_frame - 1)
            frame_end = min(total_frames - 1, end_frame - 1)
            segment_length = frame_end - frame_start + 1
        else:
            frame_start = 0
            frame_end = total_frames - 1
            segment_length = total_frames
        
        # Sample frames uniformly within the trimmed segment
        if segment_length <= self.num_frames:
            indices = [frame_start + i for i in range(segment_length)]
        else:
            indices = np.linspace(frame_start, frame_end, self.num_frames, dtype=int).tolist()
        
        for idx in indices:
            cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
            ret, frame = cap.read()
            if ret:
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                frames.append(frame_rgb)
        cap.release()
        
        while len(frames) < self.num_frames:
            frames.append(frames[-1] if frames else np.zeros((self.image_size, self.image_size, 3), dtype=np.uint8))
        
        return frames[:self.num_frames]
    
    def load_landmarks(self, video_path):
        video_id = Path(video_path).stem
        
        if video_id in self.cached_landmarks:
            landmarks = np.load(self.cached_landmarks[video_id])
            if len(landmarks) < self.num_frames:
                padding = np.tile(landmarks[-1:], (self.num_frames - len(landmarks), 1))
                landmarks = np.vstack([landmarks, padding])
            return landmarks[:self.num_frames]
        
        # Return zeros if no cached landmarks
        return np.zeros((self.num_frames, 162), dtype=np.float32)
    
    def __getitem__(self, idx):
        video_path = self.video_paths[idx]
        label = self.labels[idx]
        
        # Get frame trimming info if available
        start_frame, end_frame = None, None
        if self.frame_info is not None:
            start_frame, end_frame = self.frame_info[idx]
        
        frames = self.load_video_frames(video_path, start_frame, end_frame)
        pixel_values = self.videomae_processor(
            list(frames), 
            return_tensors="pt"
        ).pixel_values.squeeze(0)
        
        landmarks = self.load_landmarks(video_path)
        
        return {
            'pixel_values': pixel_values,
            'landmarks': torch.tensor(landmarks, dtype=torch.float32),
            'label': torch.tensor(label, dtype=torch.long)
        }

print("✓ Dataset class defined (with frame trimming support)")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# TRAINER WITH CHECKPOINT SUPPORT
# ═══════════════════════════════════════════════════════════════════════════════

class HybridASLTrainer:
    """Trainer with checkpoint resume support."""
    
    def __init__(self, model, train_loader, val_loader, device='cuda',
                 learning_rate=1e-4, weight_decay=0.01, checkpoint_dir='checkpoints'):
        
        self.model = model.to(device)
        self.device = device
        self.train_loader = train_loader
        self.val_loader = val_loader
        self.checkpoint_dir = Path(checkpoint_dir)
        self.checkpoint_dir.mkdir(parents=True, exist_ok=True)
        
        videomae_params = list(model.visual_encoder.videomae.parameters())
        other_params = [p for n, p in model.named_parameters() if 'videomae' not in n]
        
        self.optimizer = torch.optim.AdamW([
            {'params': videomae_params, 'lr': learning_rate * 0.1},
            {'params': other_params, 'lr': learning_rate}
        ], weight_decay=weight_decay)
        
        self.scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            self.optimizer, T_max=50, eta_min=1e-6
        )
        
        self.criterion = nn.CrossEntropyLoss()
        self.start_epoch = 0
        self.best_val_acc = 0
    
    def save_checkpoint(self, epoch, val_acc, filename=None):
        if filename is None:
            filename = f'checkpoint_epoch_{epoch+1}.pth'
        
        checkpoint = {
            'epoch': epoch,
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            'scheduler_state_dict': self.scheduler.state_dict(),
            'best_val_acc': self.best_val_acc,
            'val_acc': val_acc,
        }
        
        checkpoint_path = self.checkpoint_dir / filename
        torch.save(checkpoint, checkpoint_path)
        return checkpoint_path
    
    def load_checkpoint(self, checkpoint_path):
        print(f"Loading checkpoint: {checkpoint_path}")
        checkpoint = torch.load(checkpoint_path, map_location=self.device)
        
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        self.scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        self.start_epoch = checkpoint['epoch'] + 1
        self.best_val_acc = checkpoint['best_val_acc']
        
        print(f"✓ Resumed from epoch {self.start_epoch}, best val acc: {self.best_val_acc:.2f}%")
        return self.start_epoch
        
    def train_epoch(self):
        self.model.train()
        total_loss = 0
        correct = 0
        total = 0
        
        for batch in self.train_loader:
            pixel_values = batch['pixel_values'].to(self.device)
            landmarks = batch['landmarks'].to(self.device)
            labels = batch['label'].to(self.device)
            
            self.optimizer.zero_grad()
            logits = self.model(pixel_values, landmarks)
            loss = self.criterion(logits, labels)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(self.model.parameters(), 1.0)
            self.optimizer.step()
            
            total_loss += loss.item()
            _, predicted = logits.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
        
        return total_loss / len(self.train_loader), 100. * correct / total
    
    @torch.no_grad()
    def evaluate(self):
        self.model.eval()
        total_loss = 0
        correct = 0
        correct_top5 = 0
        total = 0
        
        for batch in self.val_loader:
            pixel_values = batch['pixel_values'].to(self.device)
            landmarks = batch['landmarks'].to(self.device)
            labels = batch['label'].to(self.device)
            
            logits = self.model(pixel_values, landmarks)
            loss = self.criterion(logits, labels)
            
            total_loss += loss.item()
            _, predicted = logits.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            # Top-5 accuracy
            _, top5_pred = logits.topk(5, dim=1)
            correct_top5 += sum(labels[i] in top5_pred[i] for i in range(labels.size(0)))
        
        top1_acc = 100. * correct / total
        top5_acc = 100. * correct_top5 / total
        return total_loss / len(self.val_loader), top1_acc, top5_acc
    
    def train(self, num_epochs, save_path='best_hybrid_asl_model.pth', checkpoint_every=5):
        for epoch in range(self.start_epoch, num_epochs):
            train_loss, train_acc = self.train_epoch()
            val_loss, val_acc, val_top5 = self.evaluate()
            self.scheduler.step()
            
            print(f"Epoch {epoch+1}/{num_epochs}")
            print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.2f}%")
            print(f"  Val Loss: {val_loss:.4f}, Val Top-1: {val_acc:.2f}%, Val Top-5: {val_top5:.2f}%")
            
            if val_acc > self.best_val_acc:
                self.best_val_acc = val_acc
                torch.save(self.model.state_dict(), save_path)
                print(f"  ✓ New best model saved! ({val_acc:.2f}%)")
            
            if (epoch + 1) % checkpoint_every == 0:
                ckpt_path = self.save_checkpoint(epoch, val_acc)
                print(f"  ✓ Checkpoint saved: {ckpt_path}")
            
            print()
        
        self.save_checkpoint(num_epochs - 1, val_acc, 'checkpoint_final.pth')
        return self.best_val_acc

print("✓ Trainer class defined")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# LOAD DATASET
# ═══════════════════════════════════════════════════════════════════════════════

def load_wlasl_dataset(data_dir, subset_file='nslt_300.json'):
    """Load WLASL dataset with frame trimming info."""
    data_dir = Path(data_dir)
    videos_dir = data_dir / 'videos'
    
    # Load missing videos for faster filtering
    missing_videos = set()
    missing_path = data_dir / 'missing.txt'
    if missing_path.exists():
        with open(missing_path, 'r') as f:
            missing_videos = {line.strip() for line in f if line.strip()}
        print(f"Loaded {len(missing_videos)} entries from missing.txt")
    
    with open(data_dir / subset_file, 'r') as f:
        subset_data = json.load(f)
    
    with open(data_dir / 'WLASL_v0.3.json', 'r') as f:
        wlasl_data = json.load(f)
    
    video_id_to_gloss = {}
    for entry in wlasl_data:
        gloss = entry['gloss']
        for instance in entry['instances']:
            video_id_to_gloss[instance['video_id']] = gloss
    
    idx_to_gloss = {}
    for video_id, info in subset_data.items():
        class_idx = info['action'][0]
        if video_id in video_id_to_gloss:
            gloss = video_id_to_gloss[video_id]
            if class_idx not in idx_to_gloss:
                idx_to_gloss[class_idx] = gloss
    
    video_paths = []
    labels = []
    frame_info = []  # (start_frame, end_frame) for each video
    skipped = 0
    
    for video_id, info in subset_data.items():
        # Skip if in missing.txt
        if video_id in missing_videos:
            skipped += 1
            continue
            
        video_path = videos_dir / f"{video_id}.mp4"
        if not video_path.exists():
            skipped += 1
            continue
        
        # action format: [class_idx, start_frame, end_frame]
        video_paths.append(str(video_path))
        labels.append(info['action'][0])
        frame_info.append((info['action'][1], info['action'][2]))
    
    print(f"Found {len(video_paths)} videos, {len(idx_to_gloss)} classes")
    if skipped > 0:
        print(f"Skipped {skipped} missing videos")
    print(f"Frame trimming: ENABLED (using start_frame/end_frame from annotations)")
    
    # Save label mapping
    with open('label_mapping.json', 'w') as f:
        json.dump({str(k): v for k, v in idx_to_gloss.items()}, f, indent=2)
    
    return video_paths, labels, idx_to_gloss, frame_info

# Load dataset
video_paths, labels, idx_to_gloss, frame_info = load_wlasl_dataset(
    CONFIG['data_dir'], 
    CONFIG['subset']
)
num_classes = len(idx_to_gloss)
print(f"Number of classes: {num_classes}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CREATE DATALOADERS
# ═══════════════════════════════════════════════════════════════════════════════

print("Creating datasets...")

videomae_processor = VideoMAEImageProcessor.from_pretrained('MCG-NJU/videomae-base')

full_dataset = HybridASLDataset(
    video_paths=video_paths,
    labels=labels,
    videomae_processor=videomae_processor,
    num_frames=CONFIG['num_frames'],
    landmarks_dir=CONFIG['landmarks_dir'],
    frame_info=frame_info  # Pass frame trimming info
)

# 80/20 train/val split
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size

train_dataset, val_dataset = random_split(
    full_dataset, [train_size, val_size],
    generator=torch.Generator().manual_seed(42)
)

train_loader = DataLoader(
    train_dataset, 
    batch_size=CONFIG['batch_size'], 
    shuffle=True,
    num_workers=2,
    pin_memory=True
)

val_loader = DataLoader(
    val_dataset, 
    batch_size=CONFIG['batch_size'], 
    shuffle=False,
    num_workers=2,
    pin_memory=True
)

print(f"✓ Train samples: {len(train_dataset)}")
print(f"✓ Val samples: {len(val_dataset)}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# CREATE MODEL
# ═══════════════════════════════════════════════════════════════════════════════

print("Creating model...")

model = HybridASLModel(
    num_classes=num_classes,
    videomae_model='MCG-NJU/videomae-base',
    hidden_dim=CONFIG['hidden_dim'],
    freeze_videomae=False,
    fusion_type=CONFIG['fusion_type'],
    dropout=0.3
)

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"✓ Total parameters: {total_params:,}")
print(f"✓ Trainable parameters: {trainable_params:,}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# TRAIN WITH CHECKPOINT RESUME
# ═══════════════════════════════════════════════════════════════════════════════

trainer = HybridASLTrainer(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    device=CONFIG['device'],
    learning_rate=CONFIG['learning_rate'],
    checkpoint_dir=CONFIG['checkpoint_dir']
)

# Check for existing checkpoint to resume from
checkpoint_path = Path(CONFIG['checkpoint_dir']) / 'checkpoint_final.pth'
if not checkpoint_path.exists():
    # Try to find the latest periodic checkpoint
    checkpoints = list(Path(CONFIG['checkpoint_dir']).glob('checkpoint_epoch_*.pth'))
    if checkpoints:
        # Sort by epoch number and get the latest
        checkpoint_path = max(checkpoints, key=lambda p: int(p.stem.split('_')[-1]))

if checkpoint_path.exists():
    print(f"Found existing checkpoint: {checkpoint_path}")
    trainer.load_checkpoint(checkpoint_path)
else:
    print("No checkpoint found, starting from scratch")

print("\n" + "="*70)
print("STARTING TRAINING")
print("="*70 + "\n")

best_acc = trainer.train(
    num_epochs=CONFIG['num_epochs'],
    save_path='/kaggle/working/best_hybrid_asl_model.pth',
    checkpoint_every=5
)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════════════
# TRAINING COMPLETE
# ═══════════════════════════════════════════════════════════════════════════════

print("="*70)
print("TRAINING COMPLETE!")
print("="*70)
print(f"Best validation Top-1 accuracy: {best_acc:.2f}%")
print(f"Model saved to: /kaggle/working/best_hybrid_asl_model.pth")
print(f"Checkpoints saved to: {CONFIG['checkpoint_dir']}/")
print(f"Label mapping saved to: label_mapping.json")

# List output files
print("\nOutput files:")
for f in Path('/kaggle/working').glob('*'):
    if f.is_file():
        size_mb = f.stat().st_size / (1024 * 1024)
        print(f"  {f.name}: {size_mb:.2f} MB")